# Título

In [27]:
import sys
import os
sys.path.append('../..')

# Data Manipulation
import pandas as pd
import openpyxl

In [28]:
DATA_PATH = "../../data/"

## 1. Arquivos de Layout

### 1.1. Censo 2010

In [29]:
# Carrega o arquivo de labels como um arquivo Excel
layout_2010 = pd.ExcelFile('2010_Layout_microdados_Amostra.xlsx')

In [60]:
# Funções Auxiliares

def remove_unnamed_columns(df) -> pd.DataFrame:
    """
    Remove colunas com nome Unnamed. Estas colunas vem de colunas que foram mescladas no .xlsx original.

    Args:
        df (pd.DataFrame): DataFrame com colunas Unnamed

    Returns:
        pd.DataFrame: DataFrame sem colunas Unnamed
    
    """
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

def split_col_names_values(col: pd.Series): 
    df = pd.DataFrame(columns = ['NOME', 'CHAVE', 'VALOR'])
    count = 0
    for entry in col:
        df_aux = pd.DataFrame(columns = ['NOME', 'CHAVE', 'VALOR'])
        split = entry.split('\n')
        # df_aux['NOME'] = pd.concat([df_aux['NOME'], pd.Series(split[0])])
        if len(split) > 1:
            # series_values = pd.concat([series_values, pd.Series(get_values(split))])
            # df_line = pd.DataFrame(columns = ['NOME', 'CHAVES', 'VALORES'])
            # # df_line['NOME'] = pd.Series(split[0])
            # df_line = pd.Series(get_values)
            # dct = pd.Series(get_values)
            lst = [{'CHAVE': key, 'VALOR': value} for key, value in get_values(split).items()]
            df_aux = pd.DataFrame(lst)
            df_aux['NOME'] = split[0]
        else:
            df_aux['NOME'] = pd.Series(split[0])
        df = pd.concat([df, df_aux])
    return df


def get_values(split: list):
    values = {}
    for val in range(1, len(split)):
        aux = split[val].split('-')
        if len(aux) == 1:
            aux.append(aux[0])
            aux[0] = ' '    
        # aux[1] = aux[1].replace(' ', '')
        values[aux[0]] = aux[1]
    # print(values)
    return values

#### 1.1.1. Amostra de Domicílios

In [32]:
# Carrega as labels referentes aos microdados de domicílios
df_domi = pd.read_excel(layout_2010, sheet_name='DOMI')
df_domi = remove_unnamed_columns(df_domi)
df_domi.head()

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N


#### Outros

In [33]:

# Dataframe de pessoas
df_pess = pd.read_excel(layout_2010, sheet_name='PESS')
# Dataframe de emigração
df_emig = pd.read_excel(layout_2010, sheet_name='EMIG')
# Dataframe de mortalidade
df_mort = pd.read_excel(layout_2010, sheet_name='MORT')

In [43]:

data = {
    '1': 'a',
    '4': 'b',
    'c': 'v'
}

# Convert dictionary items to a list of dictionaries
list_of_dicts = [{'key': k, 'value': v} for k, v in data.items()]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(list_of_dicts)
df

,key,value
0,1,a
1,4,b
2,c,v
